In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv


In [155]:
train = pd.read_csv("../input/nlp-getting-started/train.csv")
test = pd.read_csv("../input/nlp-getting-started/test.csv")

In [156]:
import re, string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score
from nltk.stem import PorterStemmer
import gensim
from gensim import models
from gensim.models import Word2Vec, KeyedVectors

In [157]:
df = train

In [158]:
def text_cleaning(text):
    text = text.lower() # seteamos el tweet a lowercase
    text = re.sub(r"http\S+","",text) # eliminamos los links
    text = text.translate(str.maketrans('', '', string.punctuation)) # eliminamos punctuation
    
    stop_words = set(stopwords.words('english')) # eliminamos los stop words
    word_token = word_tokenize(text)
    filtered_sentence = [w for w in word_token if not w in stop_words]
    text = ' '.join(filtered_sentence)
    
    text = ''.join(i for i in text if not i.isdigit()) # eliminamos los numeros 
    
    return text

In [159]:
# Limpiamos el keyword
df.keyword = df.keyword.apply(lambda x: text_cleaning(x) if type(x) == str else x)
df = df.replace(np.nan, '', regex=True)
df.keyword = df.keyword.str.replace('%20', '_')

In [160]:
# Limpiamos el tweet
df.text = df.text.apply(lambda x: text_cleaning(x) if type(x) == str else x)

In [161]:
df['text'] = df.apply(lambda x : x['keyword'] + x['text'] if type(x['keyword']) != '' else x['text'],axis=1) 

In [162]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = model_selection.train_test_split(df['text'],df['target'],test_size=0.33)

In [168]:
wnl = WordNetLemmatizer()
ps = PorterStemmer()
text_reconstruct_train = ''
tweet_array_train = []

for tweet in X_train:
    words_in_tweet = [wnl.lemmatize(ps.stem(word)) for word in tweet.split(' ')]
    tweet_processed = ' '.join(words_in_tweet)
    text_reconstruct_train = text_reconstruct_train + tweet_processed
    tweet_array_train.append(words_in_tweet)

In [166]:
wnl = WordNetLemmatizer()
ps = PorterStemmer()
text_reconstruct_test = ''
tweet_array_test = []

for tweet in X_test:
    words_in_tweet = [wnl.lemmatize(ps.stem(word)) for word in tweet.split(' ')]
    tweet_processed = ' '.join(words_in_tweet)
    text_reconstruct_test = text_reconstruct_test + tweet_processed
    tweet_array_test.append(words_in_tweet)

In [169]:
model = Word2Vec(tweet_array_train, min_count=1,size= 300,workers=3, window =3, sg = 1)

In [151]:
def average_w2v(list_of_tokens, model_base, dimentions = 300):
    if len(list_of_tokens) < 1:
        return np.zeros(dimentions)
    else:
        vector = [model_base[item] if item in model_base else np.zeros(dimentions) for item in list_of_tokens]
    total_length = len(vector)
    sum_of_vectors = np.sum(vector, axis=0)
    average = np.divide(sum_of_vectors, total_length)
    return average

In [152]:
def word2vec_mapping(model_base, tweets):
    mapping = tweets.apply(lambda x: average_w2v(x, model_base))
    return list(mapping)

In [171]:
mapped_train = word2vec_mapping(model, X_train)
mapped_test = word2vec_mapping(model, X_test)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [172]:
from sklearn.svm import SVC
classifier_SVM = SVC(C = 2,probability = True)
classifier_SVM.fit(mapped_train, y_train)
y_predicted_SVM = classifier_SVM.predict(mapped_test)

In [178]:
classifier_SVM.score(mapped_train,y_train),classifier_SVM.score(mapped_test,y_test)

(0.5682352941176471, 0.5746120175089534)

**Submit Kaggle**